In [ ]:
pip install --upgrade --force-reinstall scipy==1.10.1


In [1]:
!pip install ultralytics --no-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.1 MB/s eta 0:00:00a 0:00:01


**FineTune RT_DETR on my Dataset******

In [ ]:
# ✅ RT-DETR CCTV Fine-tuning Script (Weapons + Violence Detection)
# ✅ With Fixed Random Seed for Reproducibility

import torch
import random
import numpy as np
from ultralytics import RTDETR
import albumentations as A
from albumentations.pytorch import ToTensorV2
from ultralytics.data.augment import Albumentations

# ===============================
# ✅ Set Reproducibility Seed
# ===============================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # Make PyTorch deterministic
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True, warn_only=True)

    print(f"🔒 Random seed set to: {seed}")

set_seed(42)
print("✅ Using device:", "cuda" if torch.cuda.is_available() else "cpu")

# ===============================
# ✅ Custom Albumentations pipeline for CCTV-style augmentations
# ===============================
def custom_cctv_aug():
    return A.Compose([
        # Keep full image — just resize, don’t crop out small objects
        A.Resize(height=640, width=640, p=1.0),
        A.HorizontalFlip(p=0.5),

        # Slight geometric variation only
        A.Affine(scale=(0.96, 1.04), rotate=(-4, 4), shear=(-2, 2), p=0.4),

        # Moderate lighting adjustments
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.4),
        A.RandomGamma(gamma_limit=(90, 110), p=0.25),
        A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=10, val_shift_limit=10, p=0.25),
        A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.15),

        # Very light blur to simulate low-quality CCTV
        A.GaussianBlur(blur_limit=(3, 3), p=0.05),

        ToTensorV2()
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.1))

# ===============================
# ✅ Initialize Model
# ===============================
model = RTDETR("rtdetr-l.pt")

# ✅ Add Albumentations pipeline
model.add_callback('on_preprocess_batch', Albumentations(custom_cctv_aug()))

# ===============================
# ✅ Train Model
# ===============================
model.train(
    data="/kaggle/input/v-weapon-dataset/data.yaml",
    epochs=150,
    batch=8,
    imgsz=640,
    workers=2,
    device=0,
    optimizer='AdamW',
    lr0=3e-4,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0004,
    warmup_epochs=3,
    patience=20,
    cos_lr=True,
    amp=True,
    cache=True,
    rect=False,
    mosaic=0.25,
    mixup=0.3,
    seed=42,
    project="/kaggle/working/rtdetr_results",
    name="rtdetr_cctv_mixup_mosaic_adamw_seeded",
    exist_ok=True,
    save=True
)

# ===============================
# ✅ Validate Model
# ===============================
metrics = model.val()
print("📊 Validation Results:", metrics)

# ===============================
# ✅ Export to ONNX
# ===============================
model.export(format="onnx", dynamic=True)


**Resume the code from the same epoch where stop**

In [2]:
# ✅ Resume RT-DETR Training from last.pt (CCTV Weapons + Violence Detection)

import torch
import random
import numpy as np
from ultralytics import RTDETR
import albumentations as A
from albumentations.pytorch import ToTensorV2
from ultralytics.data.augment import Albumentations

# ===============================
# ✅ Set Reproducibility Seed
# ===============================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    # Make PyTorch deterministic
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True, warn_only=True)
    
    print(f"🔒 Random seed set to: {seed}")

set_seed(42)
print("✅ Using device:", "cuda" if torch.cuda.is_available() else "cpu")

# ===============================
# ✅ Custom Albumentations pipeline (same as before)
# ===============================
def custom_cctv_aug():
    return A.Compose([
        A.Resize(height=640, width=640, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.Affine(scale=(0.96, 1.04), rotate=(-4, 4), shear=(-2, 2), p=0.4),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.4),
        A.RandomGamma(gamma_limit=(90, 110), p=0.25),
        A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=10, val_shift_limit=10, p=0.25),
        A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.15),
        A.GaussianBlur(blur_limit=(3, 3), p=0.05),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.1))

# ===============================
# ✅ Load model from last checkpoint
# ===============================
# Use your saved last.pt path here
checkpoint_path = "/kaggle/input/rt-detr-lat-wight/rtdetr_cctv_mixup_mosaic_adamw_seeded/weights/last.pt"

model = RTDETR(checkpoint_path)

# Add the same Albumentations pipeline
model.add_callback('on_preprocess_batch', Albumentations(custom_cctv_aug()))

# ===============================
# ✅ Resume Training
# ===============================
# Keep all your previous training hyperparameters
model.train(
    data="/kaggle/input/vioelnceweapondetectiond/data.yaml",
    epochs=150,  # total epochs (training will resume from saved epoch)
    batch=8,
    imgsz=640,
    workers=2,
    device=0,
    optimizer='AdamW',
    lr0=3e-4,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0004,
    warmup_epochs=3,
    patience=20,
    cos_lr=True,
    amp=True,
    cache=True,
    rect=False,
    mosaic=0.25,
    mixup=0.3,
    seed=42,
    project="/kaggle/working/rtdetr_results",
    name="rtdetr_cctv_mixup_mosaic_adamw_seeded",
    exist_ok=True,
    save=True,    # make sure saving continues
    resume=checkpoint_path  # <-- This tells RT-DETR to continue from last.pt
)

# ===============================
# ✅ Validate Model
# ===============================
metrics = model.val()
print("📊 Validation Results:", metrics)

# ===============================
# ✅ Export to ONNX
# ===============================
model.export(format="onnx", dynamic=True)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🔒 Random seed set to: 42
✅ Using device: cuda
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
Ultralytics 8.3.223 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/input/vioelnceweapondetectiond/data.yaml, degrees=0.0, deterministic

/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


     68/150      6.83G      1.558     0.4558      2.505          8        640: 100% ━━━━━━━━━━━━ 965/965 1.7it/s 9:37<0.6s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 60/60 2.9it/s 20.6s0.3ss
                   all        948        806       0.55     0.0963      0.024    0.00579

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
     69/150      7.31G      1.073     0.7515     0.5924         14        640: 0% ──────────── 0/965  0.6s

/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


     69/150      7.31G     0.7594     0.9468     0.4564         14        640: 100% ━━━━━━━━━━━━ 965/965 1.7it/s 9:41<0.6s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 60/60 3.1it/s 19.1s0.3s
                   all        948        806      0.255      0.184      0.122     0.0404

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
     70/150      7.03G     0.7289     0.8341     0.4562         13        640: 0% ──────────── 0/965  0.6s

...

/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


    128/150      6.95G     0.4008     0.5085     0.2603         12        640: 100% ━━━━━━━━━━━━ 965/965 1.6it/s 9:47<0.6s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 60/60 3.1it/s 19.5s0.3s
                   all        948        806      0.863      0.801      0.826      0.476

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
    129/150      6.95G     0.6323     0.5323     0.3977          9        640: 0% ──────────── 0/965  0.6s

/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


    129/150      6.97G     0.3908     0.5099     0.2517          5        640: 100% ━━━━━━━━━━━━ 965/965 1.6it/s 9:47<0.6s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 60/60 3.1it/s 19.4s0.3s
                   all        948        806       0.86      0.799      0.829      0.477
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 109, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

62 epochs completed in 10.455 hours.
Optimizer stripped from /kaggle/working/rtdetr_results/rtdetr_cctv_mixup_mosaic_adamw_seeded/weights/last.pt, 66.2MB
Ultralytics 8.3.223 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
rt-detr-l summary: 302 layers, 31,987,850 parameters, 0 gradients
val: Fast image access ✅ (ping: 0.5±1.2 ms, read: 34.4±10.5 MB/s, size: 4

'/kaggle/working/rtdetr_results/rtdetr_cctv_mixup_mosaic_adamw_seeded/weights/last.onnx'

In [4]:
import shutil

# Path to the folder you want to zip
folder_path = "/kaggle/working/runs"

# Output zip file name (it will be saved in /kaggle/working/)
zip_path = "rtdetrrun"

# Create the zip file
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', folder_path)

print(f"✅ Folder successfully zipped: {zip_path}")


✅ Folder successfully zipped: rtdetrrun
